# Airbnb Analysis Project

Aim: To analyse prices and expected earnings from airbnb rental properties


In [1]:
# Import the libraries required
import pandas as pd
import numpy as np

import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

# this allows plots to appear directly in the notebook
%matplotlib inline

In [2]:
listings_data2 = pd.read_csv("sydney/oct15/listings2.csv", )

In [3]:
listings_data2.head(2)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,5309357,https://www.airbnb.com/rooms/5309357,2.015100e+13,2/10/15,Character Waterfront CottageRetreat,Water front & access character 2 BR Cottage on...,NaN,Water front & access character 2 BR Cottage on...,none,NaN,...,8,f,NaN,NaN,f,flexible,f,f,2,0.26
1,5388735,https://www.airbnb.com/rooms/5388735,2.015100e+13,2/10/15,3 B/R- $AU90 Each,All rooms with built in and sleeps 2 comfortab...,3 Spacious BRs with built ins and Beds.$AU90 f...,All rooms with built in and sleeps 2 comfortab...,none,All families living in our (website hidden) is...,...,NaN,f,NaN,NaN,f,strict,f,f,1,NaN


In [4]:
#clean the data   - need to remove:  amenities

In [5]:
listings_data2_clean = listings_data2.drop(['amenities' , 'neighbourhood_group_cleansed', 'listing_url', 'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 'host_verifications', 'host_has_profile_pic', 'host_identity_verified', 'street', 'neighbourhood', 'neighbourhood_group_cleansed', 'state', 'market', 'smart_location', 'country_code', 'country', 'square_feet', 'security_deposit', 'cleaning_fee', 'calendar_last_scraped', 'first_review', 'last_review', 'requires_license', 'license', 'jurisdiction_names', 'require_guest_profile_picture', 'require_guest_phone_verification', 'scrape_id', 'last_scraped' , 'name', 'summary' , 'space' , 'description' , 'experiences_offered' , 'neighborhood_overview' , 'notes' , 'transit' , 'thumbnail_url' , 'medium_url' , 'picture_url' , 'xl_picture_url' , 'host_id' , 'host_url' , 'host_name' , 'host_since' , 'host_location' , 'host_about'], 1)

In [6]:
# Remove horrible pricing format

listings_data2_clean['price'] = listings_data2_clean['price'].str.replace('$', '').str.replace(',', '').str.rstrip('00').str.rstrip('.')
listings_data2_clean['weekly_price'] = listings_data2_clean['weekly_price'].str.replace('$', '').str.replace(',', '').str.rstrip('00').str.rstrip('.')
listings_data2_clean['monthly_price'] = listings_data2_clean['monthly_price'].str.replace('$', '').str.replace(',', '').str.rstrip('00').str.rstrip('.')
listings_data2_clean['extra_people'] = listings_data2_clean['extra_people'].str.replace('$', '').str.replace(',', '').str.rstrip('00').str.rstrip('.')

In [7]:
# converts strings to int
listings_data2_clean['price'] = listings_data2_clean['price'].astype(int)

#gets a list of daily prices times by 7
weekly = listings_data2_clean['price']*int('7')
listings_data2_clean['weekly_price'] = listings_data2_clean['weekly_price'].fillna(weekly).astype(int)

#gets a list of weekly prices times by 30.4 / 7 
monthly = listings_data2_clean['weekly_price']*float('4.34286')
listings_data2_clean['monthly_price'] = listings_data2_clean['monthly_price'].fillna(monthly).astype(int)

# Clean Response Rate
listings_data2_clean['host_response_rate'] = listings_data2_clean['host_response_rate'].str.rstrip('%').fillna(0).astype(int)
listings_data2_clean['host_acceptance_rate'] = listings_data2_clean['host_acceptance_rate'].str.rstrip('%').fillna(0).astype(int)

In [8]:
listings_data2_clean['host_is_superhost'] = listings_data2_clean['host_is_superhost'].str.replace('t', '1').str.replace('f', '0').fillna(0).astype(int)
listings_data2_clean['is_location_exact'] = listings_data2_clean['is_location_exact'].str.replace('t', '1').str.replace('f', '0').fillna(0).astype(int)
listings_data2_clean['has_availability'] = listings_data2_clean['has_availability'].str.replace('t', '1').str.replace('f', '0').fillna(0).astype(int)
listings_data2_clean['instant_bookable'] = listings_data2_clean['instant_bookable'].str.replace('t', '1').str.replace('f', '0').fillna(0).astype(int)


In [9]:

#listings_data2_clean['zipcode'] = listings_data2_clean['zipcode']. fillna(0).astype(int)


In [10]:
# Classification Variables

#listings_data2_clean['property_type'] = listings_data2_clean['property_type'].map({'Apartment':1, 'Bed & Breakfast':2, 'Boat':3, 'Bungalow':4, 'Cabin':5, 'Camper/RV':6, 'Castle':7, 'Chalet':8, 'Condominium':9, 'Dorm':10, 'Earth House':11, 'House':12, 'Island':13, 'Loft ':14, 'Other':15, 'Parking Space':16, 'Tent':17, 'Tipi ':18, 'Townhouse':19, 'Treehouse':20, 'Villa':21})
#listings_data2_clean['room_type'] = listings_data2_clean['room_type'].map({'Shared room':1, 'Private room':2, 'Entire home/apt':3})
#listings_data2_clean['cancellation_policy'] = listings_data2_clean['cancellation_policy'].map({'strict':1, 'moderate':2, 'flexible':3})
#listings_data2_clean['bed_type'] = listings_data2_clean['bed_type'].map({'Real Bed':5, 'Pull-out Sofa':4, 'Futon':3, 'Couch':2, 'Airbed':1})


In [11]:
# Cleaning Classification types
#listings_data2_clean['room_type'] = listings_data2_clean['room_type'].fillna(0).astype(int)
#listings_data2_clean['property_type'] = listings_data2_clean['property_type'].fillna(0).astype(int)
#listings_data2_clean['cancellation_policy'] = listings_data2_clean['cancellation_policy'].fillna(0).astype(int)
#listings_data2_clean['bed_type'] = listings_data2_clean['bed_type'].fillna(0).astype(int)


In [12]:
listings_data2_clean = listings_data2_clean.fillna(0)

In [13]:
room_dummies = pd.get_dummies(listings_data2_clean.room_type, prefix='Room').iloc[:, 1:]
property_dummies = pd.get_dummies(listings_data2_clean.property_type, prefix='Property').iloc[:, 1:]
cancellation_dummies = pd.get_dummies(listings_data2_clean.cancellation_policy, prefix='Cancellation').iloc[:, 1:]
bed_dummies = pd.get_dummies(listings_data2_clean.bed_type, prefix='Bed').iloc[:, 1:]

In [14]:
listings_data2_clean = pd.concat([listings_data2_clean, room_dummies], axis=1)
listings_data2_clean = pd.concat([listings_data2_clean, property_dummies], axis=1)
listings_data2_clean = pd.concat([listings_data2_clean, cancellation_dummies], axis=1)
listings_data2_clean = pd.concat([listings_data2_clean, bed_dummies], axis=1)


In [15]:
listings_data2_final = listings_data2_clean

In [16]:
# list(listings_data2_final)

In [17]:

# create a list of feature columns   - without amenities
feature_cols = ['host_response_rate' , 'host_acceptance_rate' ,  'availability_365'   , 'calculated_host_listings_count' , 'reviews_per_month' , 'availability_30' , 'availability_60' , 'availability_90', 'review_scores_rating' , 'latitude' , 'longitude' , 'review_scores_accuracy' , 'review_scores_cleanliness' , 'review_scores_checkin' , 'review_scores_communication' , 'review_scores_location' , 'review_scores_value' , 'number_of_reviews',  'accommodates' , 'bathrooms' , 'bedrooms' , 'beds', 'instant_bookable', 'Room_Private room',
 'Room_Shared room', 'Property_Apartment', 'Property_Bed & Breakfast', 'Property_Boat', 'Property_Bungalow', 'Property_Cabin', 'Property_Camper/RV', 'Property_Castle', 'Property_Chalet', 'Property_Condominium', 'Property_Dorm', 'Property_Earth House', 'Property_House', 'Property_Island', 'Property_Loft', 'Property_Other', 'Property_Parking Space', 'Property_Tent', 'Property_Tipi', 'Property_Townhouse', 'Property_Treehouse', 'Property_Villa', 'Cancellation_moderate', 'Cancellation_strict', 'Cancellation_super_strict_60', 'Bed_Couch', 'Bed_Futon', 'Bed_Pull-out Sofa', 'Bed_Real Bed'
]


In [18]:
y = listings_data2_final['price']
X = listings_data2_final[[ 'host_response_rate' , 'host_acceptance_rate' ,  'availability_365'  , 'calculated_host_listings_count' , 'reviews_per_month' , 'availability_30' , 'availability_60' , 'availability_90', 'review_scores_rating' , 'latitude' , 'longitude' , 'review_scores_accuracy' , 'review_scores_cleanliness' , 'review_scores_checkin' , 'review_scores_communication' , 'review_scores_location' , 'review_scores_value' , 'number_of_reviews',  'accommodates' , 'bathrooms' , 'bedrooms' , 'beds', 'instant_bookable', 'Room_Private room',
 'Room_Shared room', 'Property_Apartment', 'Property_Bed & Breakfast', 'Property_Boat', 'Property_Bungalow', 'Property_Cabin', 'Property_Camper/RV', 'Property_Castle', 'Property_Chalet', 'Property_Condominium', 'Property_Dorm', 'Property_Earth House', 'Property_House', 'Property_Island', 'Property_Loft', 'Property_Other', 'Property_Parking Space', 'Property_Tent', 'Property_Tipi', 'Property_Townhouse', 'Property_Treehouse', 'Property_Villa', 'Cancellation_moderate', 'Cancellation_strict', 'Cancellation_super_strict_60', 'Bed_Couch', 'Bed_Futon', 'Bed_Pull-out Sofa', 'Bed_Real Bed' ]]





In [19]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
# Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X_train, y_train)
lm.coef_

In [ ]:

# make predictions and evaluate
import numpy as np
from sklearn import metrics
preds = lm.predict(X_test)

np.sqrt(metrics.mean_squared_error(y_test, preds))
lm.score(X_test, y_test) 

In [ ]:
########## Ridge Regression Model ##########
# ridge regression (alpha must be positive, larger means more regularization)
from sklearn.linear_model import Ridge
rreg = Ridge(alpha=0.1, normalize=True)
rreg.fit(X_train, y_train)
rreg.coef_
preds = rreg.predict(X_test)
np.sqrt(metrics.mean_squared_error(y_test, preds))
# Is this model better? Why?
rreg.score(X_test, y_test) 




In [ ]:
preds = rfreg.predict(X)
preds = pd.DataFrame(preds)
result = pd.concat([y, preds], axis=1, ignore_index=True)
result.head(10)

In [ ]:
# use RidgeCV to select best alpha
from sklearn.linear_model import RidgeCV
alpha_range = 10.**np.arange(-4, 3)
rregcv = RidgeCV(normalize=True, scoring='mean_squared_error', alphas=alpha_range)
rregcv.fit(X_train, y_train)
rregcv.alpha_
preds = rregcv.predict(X_test)
np.sqrt(metrics.mean_squared_error(y_test, preds))
# What is the range of alp vlues we are searching over?
rregcv.score(X_test, y_test) 

In [ ]:
y_test.head(10)

In [ ]:
preds[0:10]

In [ ]:
########## Elastic Net Regression Model ##########
# Elastic (alpha must be positive, larger means more regularization)
from sklearn.linear_model import ElasticNet
els = ElasticNet(alpha=0.01, normalize=True)
els.fit(X_train, y_train)
els.coef_
preds = els.predict(X_test)
np.sqrt(metrics.mean_squared_error(y_test, preds))
els.score(X_test, y_test) 

In [ ]:
# try a smaller alpha
els = ElasticNet(alpha=0.001, normalize=True)
els.fit(X_train, y_train)
els.coef_
preds = els.predict(X_test)
np.sqrt(metrics.mean_squared_error(y_test, preds))
els.score(X_test, y_test) 

In [ ]:
# use LassoCV to select best alpha (tries 100 alphas by default)
from sklearn.linear_model import ElasticNetCV
elscv = ElasticNetCV(normalize=True, alphas=alpha_range)
elscv.fit(X_train, y_train)
elscv.alpha_
elscv.coef_
preds = elscv.predict(X_test)
np.sqrt(metrics.mean_squared_error(y_test, preds))
elscv.score(X_test, y_test) 

In [ ]:
#SVM Support Vecor Machine ( Not Working!!!)

In [ ]:

#from sklearn import svm
#clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)


In [ ]:
#clf.score(X_test, y_test) 

In [ ]:
#from sklearn import metrics
#svc = svm.SVC(kernel='linear', C=1)
#cvsvm = cross_validation.cross_val_score(svc, X, y, cv=5)
#scores

In [20]:
# Random Forest

In [21]:
# import class, instantiate estimator, fit with all data
from sklearn.ensemble import RandomForestRegressor
rfreg = RandomForestRegressor(n_estimators=300, max_features=11, oob_score=True, random_state=1)
rfreg.fit(listings_data2_final[feature_cols], listings_data2_final.price)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=11, max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=300, n_jobs=1, oob_score=True, random_state=1,
           verbose=0, warm_start=False)

In [22]:
s_rfreg = rfreg.oob_score_


0.4175214525036044

0.92123602925857362

In [23]:
Xreg = rfreg.predict(X)
Xreg = pd.DataFrame(Xreg)
result = pd.concat([y, Xreg], axis=1, ignore_index=True)
result.head(20)


,0,1
0,295,308.836667
1,89,135.210000
2,72,72.220000
3,270,332.283333
4,97,86.670000
5,129,111.276667
6,610,650.776667
7,80,85.913333
8,80,85.126667
9,65,67.086667


In [25]:
pd.DataFrame({'feature':feature_cols, 'importance':rfreg.feature_importances_})

,feature,importance
0,host_response_rate,2.521925e-02
1,host_acceptance_rate,1.730433e-02
2,availability_365,3.601597e-02
3,calculated_host_listings_count,9.124786e-03
4,reviews_per_month,1.918377e-02
5,availability_30,1.531501e-02
6,availability_60,1.811107e-02
7,availability_90,3.217028e-02
8,review_scores_rating,8.789077e-03
9,latitude,9.809789e-02


In [ ]:
# Gradient Boosting

In [26]:
from sklearn import ensemble
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error

###############################################################################
# Load data

X = X.astype(np.float32)

In [27]:



###############################################################################  'latitude' , 'longitude'
# Fit regression model
params = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 1,
          'learning_rate': 0.01, 'loss': 'ls'}
gbst = ensemble.GradientBoostingRegressor(**params)

gbst.fit(X_train, y_train)
mse = mean_squared_error(y_test, gbst.predict(X_test))
print("MSE: %.4f" % mse)

MSE: 25127.4246


In [46]:
s_gbst = gbst.score(X_test, y_test) 
s_gbst

0.62294860893326209

In [ ]:

import matplotlib.pyplot as plt

###############################################################################
# Plot training deviance

# compute test set deviance
test_score = np.zeros((params['n_estimators'],), dtype=np.float64)

for i, y_pred in enumerate(gbst.staged_predict(X_test)):
    test_score[i] = gbst.loss_(y_test, y_pred)

plt.figure(figsize=(12, 6))
plt.title('Deviance')
plt.plot(np.arange(params['n_estimators']) + 1, gbst.train_score_, 'b-',
         label='Training Set Deviance')
plt.plot(np.arange(params['n_estimators']) + 1, test_score, 'r-',
         label='Test Set Deviance')
plt.legend(loc='upper right')
plt.xlabel('Boosting Iterations')
plt.ylabel('Deviance')



In [ ]:
###############################################################################
# Plot feature importance
feature_importance = gbst.feature_importances_
# make importances relative to max importance
feature_importance = 100.0 * (feature_importance / feature_importance.max())
sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0]) + .5
plt.figure(figsize=(12, 12))


plt.barh(pos, feature_importance[sorted_idx], align='center')
plt.yticks(pos, X[sorted_idx])
plt.xlabel('Relative Importance')
plt.title('Variable Importance')
plt.show()

In [ ]:
# Gradient Boosting + Kfolds

In [31]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn import ensemble
from sklearn.cross_validation import KFold
from sklearn.cross_validation import train_test_split

## y = y.astype(np.float32).as_matrix()

X = X.astype(np.float32).as_matrix()
y = y.astype(np.float32).as_matrix()

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)



Accuracy: 0.5948


200

In [44]:
# Fit classifier with out-of-bag estimates
params = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 1,
          'learning_rate': 0.01}
gbkf = ensemble.GradientBoostingRegressor(**params)

gbkf.fit(X_train, y_train)
s_gbkf = gbkf.score(X_test, y_test)
print("Accuracy: {:.4f}".format(s_gbkf))

n_estimators = params['n_estimators']
x = np.arange(n_estimators) + 1
n_estimators

Accuracy: 0.6230


500

In [41]:
def heldout_score(gbkf, X_test, y_test):
    """compute deviance scores on ``X_test`` and ``y_test``. """
    score = np.zeros((n_estimators,), dtype=np.float64)
    for i, y_pred in enumerate(gbkf.staged_decision_function(X_test)):
        score[i] = gbkf.loss_(y_test, y_pred)
    return score


def cv_estimate(n_folds=3):
    cv = KFold(n=X_train.shape[0], n_folds=n_folds)
    cv_gbkf = ensemble.GradientBoostingRegressor(**params)
    val_scores = np.zeros((n_estimators,), dtype=np.float64)
    for train, test in cv:
        cv_gbkf.fit(X_train[train], y_train[train])
        val_scores += heldout_score(cv_gbkf, X_train[test], y_train[test])
    val_scores /= n_folds
    return val_scores


# Estimate best n_estimator using cross-validation
cv_score = cv_estimate(3)

/Users/nishantkumar/anaconda/lib/python2.7/site-packages/sklearn/utils/__init__.py:93: DeprecationWarning: Function staged_decision_function is deprecated;  and will be removed in 0.19
  warnings.warn(msg, category=DeprecationWarning)
/Users/nishantkumar/anaconda/lib/python2.7/site-packages/sklearn/utils/__init__.py:93: DeprecationWarning: Function staged_decision_function is deprecated;  and will be removed in 0.19
  warnings.warn(msg, category=DeprecationWarning)
/Users/nishantkumar/anaconda/lib/python2.7/site-packages/sklearn/utils/__init__.py:93: DeprecationWarning: Function staged_decision_function is deprecated;  and will be removed in 0.19
  warnings.warn(msg, category=DeprecationWarning)


In [45]:
s_cv_gbkf = cv_gbkf.score(X_test, y_test)
print("Accuracy: {:.4f}".format(s_cv_gbkf))

Accuracy: 0.6211


In [50]:
y = pd.DataFrame(y)
X = pd.DataFrame(X)

y_reg = rfreg.predict(X)
y_gbst = gbst.predict(X)
y_gbkf = gbkf.predict(X)
y_cv_gbkf = cv_gbkf.predict(X)

y_avg = np.mean( np.array([ y_reg, y_cv_gbkf ]), axis=0 )

y_reg = pd.DataFrame(y_reg)
y_gbst = pd.DataFrame(y_gbst)
y_gbkf = pd.DataFrame(y_gbkf)
y_cv_gbkf = pd.DataFrame(y_cv_gbkf)
y_avg = pd.DataFrame(y_avg)

result = pd.concat([y, y_avg, y_reg, y_gbst, y_cv_gbkf], axis=1, ignore_index=True)
result.head(50)

,0,1,2,3,4
0,295,271.144270,308.836667,233.451874,233.451874
1,89,156.662779,135.210000,178.115559,178.115559
2,72,70.121479,72.220000,68.022958,68.022958
3,270,350.131571,332.283333,367.979810,367.979810
4,97,87.943079,86.670000,89.216158,89.216158
5,129,87.843391,111.276667,64.410116,64.410116
6,610,521.826985,650.776667,392.877303,392.877303
7,80,74.454708,85.913333,62.996082,62.996082
8,80,74.422483,85.126667,63.718299,63.718299
9,65,65.584393,67.086667,64.082120,64.082120


In [ ]:
# Compute best n_estimator for test data
test_score = heldout_score(gbkf, X_test, y_test)

# negative cumulative sum of oob improvements
cumsum = -np.cumsum(cv_gbkf.oob_improvement_)

# min loss according to OOB
oob_best_iter = x[np.argmin(cumsum)]

# min loss according to test (normalize such that first loss is 0)
test_score -= test_score[0]
test_best_iter = x[np.argmin(test_score)]

# min loss according to cv (normalize such that first loss is 0)
cv_score -= cv_score[0]
cv_best_iter = x[np.argmin(cv_score)]

# color brew for the three curves
oob_color = list(map(lambda x: x / 256.0, (190, 174, 212)))
test_color = list(map(lambda x: x / 256.0, (127, 201, 127)))
cv_color = list(map(lambda x: x / 256.0, (253, 192, 134)))

# plot curves and vertical lines for best iterations
plt.plot(x, cumsum, label='OOB loss', color=oob_color)
plt.plot(x, test_score, label='Test loss', color=test_color)
plt.plot(x, cv_score, label='CV loss', color=cv_color)
plt.axvline(x=oob_best_iter, color=oob_color)
plt.axvline(x=test_best_iter, color=test_color)
plt.axvline(x=cv_best_iter, color=cv_color)

# add three vertical lines to xticks
xticks = plt.xticks()
xticks_pos = np.array(xticks[0].tolist() +
                      [oob_best_iter, cv_best_iter, test_best_iter])
xticks_label = np.array(list(map(lambda t: int(t), xticks[0])) +
                        ['OOB', 'CV', 'Test'])
ind = np.argsort(xticks_pos)
xticks_pos = xticks_pos[ind]
xticks_label = xticks_label[ind]
plt.xticks(xticks_pos, xticks_label)

plt.legend(loc='upper right')
plt.ylabel('normalized loss')
plt.xlabel('number of iterations')

plt.show()

In [ ]:
from sklearn import cross_validation, datasets, linear_model
lasso = linear_model.Lasso()
alphas = np.logspace(-2, -0.5, 10)

scores = list()
scores_std = list()

for alpha in alphas:
    lasso.alpha = alpha
    this_scores = cross_validation.cross_val_score(lasso, X, y, n_jobs=1)
    scores.append(np.mean(this_scores))
    scores_std.append(np.std(this_scores))

plt.figure(figsize=(4, 3))
plt.semilogx(alphas, scores)
# plot error lines showing +/- std. errors of the scores
plt.semilogx(alphas, np.array(scores) + np.array(scores_std) / np.sqrt(len(X)),
             'b--')
plt.semilogx(alphas, np.array(scores) - np.array(scores_std) / np.sqrt(len(X)),
             'b--')
plt.ylabel('CV score')
plt.xlabel('alpha')
plt.axhline(np.max(scores), linestyle='--', color='.5')



In [ ]:
##############################################################################
# Bonus: how much can you trust the selection of alpha?

# To answer this question we use the LassoCV object that sets its alpha
# parameter automatically from the data by internal cross-validation (i.e. it
# performs cross-validation on the training data it receives).
# We use external cross-validation to see how much the automatically obtained
# alphas differ across different cross-validation folds.
lasso_cv = linear_model.LassoCV(alphas=alphas)
k_fold = cross_validation.KFold(len(X), 3)
y = np.array(y)
X = np.array(X)
print("Answer to the bonus question:",
      "how much can you trust the selection of alpha?")
print()
print("Alpha parameters maximising the generalization score on different")
print("subsets of the data:")
for k, (train, test) in enumerate(k_fold):
    lasso_cv.fit(X[train], y[train])
    print("[fold {0}] alpha: {1:.5f}, score: {2:.5f}".
          format(k, lasso_cv.alpha_, lasso_cv.score(X[test], y[test])))
print()
print("Answer: Not very much since we obtained different alphas for different")
print("subsets of the data and moreover, the scores for these alphas differ")
print("quite substantially.")

plt.show()

In [ ]:
# XG Boost  ( Cant Install!!!!)

In [ ]:
#from sklearn.cross_validation import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
#! pip install --upgrade xgboost

In [ ]:
#import xgboost as xgb
#from sklearn.preprocessing import LabelEncoder
#import numpy as np
#from sklearn.cross_validation import KFold, train_test_split
#from sklearn.metrics import confusion_matrix, mean_squared_error
#from sklearn.grid_search import GridSearchCV

In [ ]:
#gbm = xgb.XGBRegressor(max_depth=3, n_estimators=300, learning_rate=0.05).fit(X_train, y_train)

In [ ]:
#predictions = gbm.predict(X_test)


In [ ]:
#err = []
#kf = KFold(y.shape[0], n_folds=2, shuffle=True, random_state=rng)
#for train_index, test_index in kf:
#    xgb_model = xgb.XGBRegressor().fit(X[train_index],y[train_index])
#    predictions = xgb_model.predict(X[test_index])
#    actuals = y[test_index]
#    err.append(mean_squared_error(actuals, predictions))
#err


In [ ]:
y.head(10)

In [ ]:
from sklearn.cross_validation import KFold

def run_cv(X,y,clf_class,**kwargs):
    kf = KFold(len(y),n_folds=5,shuffle=True)
    y_pred = y.copy()
   
   # Iterate through folds
    for train_index, test_index in kf:
        X_train, X_test = X[train_index], X[test_index]
        y_train = y[train_index]
        
        clf = clf_class(**kwargs)
        clf.fit(X_train,y_train)
        y_pred[test_index] = clf.predict(X_test)
    return y_pred

In [ ]:

from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor as RF
from sklearn.neighbors import KNeighborsRegressor as KNN

def accuracy(y_true,y_pred):
    return np.mean(y_true == y_pred)



In [ ]:
print("Support vector machines:")
#print("%.3f" % accuracy(y, run_cv(X1,y,SVR)))
print("Random forest:")
print("%.3f" % accuracy(y, run_cv(X1,y,RF, "n_estimators=1000, max_features=8, oob_score=True, random_state=1")))
print("K-nearest-neighbors:")
#print("%.3f" % accuracy(y, run_cv(X1,y,KNN)))

In [ ]:
#pd.scatter_matrix(listings_data2_clean, figsize=(10,10))

In [ ]:
#Predicted price vs accomodates
#print linreg.predict([1,-33.61,151.28])

#price_pred = linreg.predict(X)
#plt.plot(listings_data2.accommodates, price_pred, color='red')

#plt.scatter(listings_data2.accommodates, listings_data2.price)


In [ ]:
# Plot reviews per month vs price

#plt.scatter(listings_data2.reviews_per_month, listings_data2.price)

In [ ]:
#install plotly

#! pip install plotly


In [ ]:
import plotly
print plotly.__version__  # version >1.9.4 required
from plotly.graph_objs import Scatter, Layout
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
init_notebook_mode()

In [ ]:
price_map = listings_data2_final[[ 'price' , 'longitude' ,  'latitude']]

In [ ]:
#color pattern
scl = [0,"rgb(150,0,90)"],[0.125,"rgb(0, 0, 200)"],[0.25,"rgb(0, 25, 255)"],\
[0.375,"rgb(0, 152, 255)"],[0.5,"rgb(44, 255, 150)"],[0.625,"rgb(151, 255, 0)"],\
[0.75,"rgb(255, 234, 0)"],[0.875,"rgb(255, 111, 0)"],[1,"rgb(255, 0, 0)"]

listings = [ dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = price_map['longitude'],
        lat = price_map['latitude'],
        hoverinfo = 'text',
        text = '$' + price_map['price'].astype(str),
        mode = 'markers',
        marker = dict(
            color = price_map['price'],
            colorscale = scl,
            cmin = 5,
            cmax = 400,
            reversescale = True,
            opacity = 0.7,
            size = 2,        
            colorbar = dict(
                thickness = 10,
                titleside = "right",
                outlinecolor = "rgba(68, 68, 68, 0)",
                ticks = "outside",
                ticklen = 3,
                showticksuffix = "last",
                tickprefix = "$",
                dtick = 0.1
            ),                
        ),
        
       )]


layout = dict(
        title = 'Airbnb Prices in NSW',
        showlegend = False,
        height = 800,
    
        geo = dict(
        scope = 'world',
        showland = True,
        landcolor = "rgb(212, 212, 212)",
        subunitcolor = "rgb(255, 255, 255)",
        countrycolor = "rgb(255, 255, 255)",
        showlakes = True,
        lakecolor = "rgb(255, 255, 255)",
        showsubunits = True,
        showcountries = True,
        resolution = 50,
        lonaxis = dict(
            showgrid = True,
            gridwidth = 0.5,
            range= [ 151.1, 151.35 ],
            dtick = 5
        ),
        lataxis = dict (
            showgrid = True,
            gridwidth = 0.5,
            range= [ -33.95, -33.75 ],
            dtick = 5
        )
        ),
    )

fig = dict( data=listings, layout=layout )

iplot(fig)